In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data
import torch.nn.functional as F
import torch.optim as optim

from models import *
from learning import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
narma_df = pd.read_csv('../NARMA10.csv', header=None)
narma_df.iloc[:, :20] # visualize the first 20 columns

In [ ]:
x_data = torch.tensor(narma_df.iloc[0].values, dtype=torch.float32) # float 32 for better memory efficiency
y_data = torch.tensor(narma_df.iloc[1].values, dtype=torch.float32)

# 4000 tr, 1000 val, 5000 test
train_x, test_x = x_data[:5000], x_data[5000:]
train_y, test_y = y_data[:5000], y_data[5000:]

train_x, val_x = train_x[:4000], train_x[4000:]
train_y, val_y = train_y[:4000], train_y[4000:]

In [ ]:
window_size = 30

train_data = SequenceDataset(train_x, train_y, window_size)
train_loader = data.DataLoader(train_data, batch_size=500, shuffle=False)

val_data = SequenceDataset(val_x, val_y, window_size)
val_loader = data.DataLoader(val_data, batch_size=len(val_data), shuffle=False)

In [ ]:
# this code is just a try to see how to plot
'''plt.plot(train_mse_history, label='Train MSE')
plt.plot(val_mse_history, label='Val MSE')
plt.legend()'''

In [ ]:
# this code is just a try to see how to plot
'''train_loader = data.DataLoader(train_data, batch_size=len(train_data), shuffle=False)
plt.figure(figsize=(20, 10))
for x, y in train_loader:
    with torch.no_grad():
        out = model(x)
        plt.plot(y.cpu().numpy(), label='True')
        plt.plot(out.cpu().numpy(), label='Predicted')
        plt.legend()
        plt.show()'''